In [1]:
# Import modules

from __future__ import print_function
import numpy as np
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
from six.moves import range

%matplotlib inline

In [2]:
# Load training and test file
flank_len = 9

train_file = 'train_f' + str(flank_len) + '.pickle'

with open(train_file, 'rb') as train:
    save = pickle.load(train)
    dataset = save['seq']  # data type : numpy.ndarray // 'list' can't be dealt with 'shape' and 'reshape' 
    labels = save['label']

    del save

test_file = 'test_f' + str(flank_len) + '.pickle'

with open(test_file, 'rb') as test:
    save = pickle.load(test)
    test_dataset = save['seq']
  
    del save

In [3]:
# Determine how many folds
fold = 4
num = int(dataset.shape[0])
        
# Split data into several parts
dataset_array = np.split(dataset, np.arange(int(np.ceil(num/fold)), num, int(np.ceil(num/fold))), axis = 0)
labels_array = np.split(labels, np.arange(int(np.ceil(num/fold)), num, int(np.ceil(num/fold))), axis = 0)

train_dataset = np.empty([1, 101+(2*flank_len), 4])
train_labels = np.empty([1,2])
for i in range(0, fold):
    if i == 0:  # (i+1)th partition
        valid_dataset = dataset_array[i]
        valid_labels = labels_array[i]
    else:
        train_dataset = np.concatenate((train_dataset, dataset_array[i]), axis = 0)
        train_labels = np.concatenate((train_labels, labels_array[i]), axis = 0)
train_dataset = np.delete(train_dataset,0,0)
train_labels = np.delete(train_labels,0,0)
print('training set', train_dataset.shape, train_labels.shape)
print('validation set', valid_dataset.shape, valid_labels.shape)
print('test set', test_dataset.shape)

training set (58148, 119, 4) (58148, 2)
validation set (19383, 119, 4) (19383, 2)
test set (19383, 119, 4)


In [4]:
dataset_row = train_dataset.shape[1]
dataset_column = train_dataset.shape[2]
num_channels = 1

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, dataset_row, dataset_column, num_channels)).astype(np.float32)
    return dataset, labels
def test_reformat(dataset):
    dataset = dataset.reshape(
        (-1, dataset_row, dataset_column, num_channels)).astype(np.float32)
    return dataset

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset = test_reformat(test_dataset)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape)

Training set (58148, 119, 4, 1) (58148, 2)
Validation set (19383, 119, 4, 1) (19383, 2)
Test set (19383, 119, 4, 1)


In [5]:
num_labels = 2
batch_size = train_dataset.shape[0] // 200
depth = 16
num_hidden = 128
num_hidden2 = 64
beta_regul = 8e-5
stride_1 = 2
stride_2 = 2

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, dataset_row, dataset_column, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [21, 4, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [9, 2, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [math.ceil(math.ceil(train_dataset.shape[1]/stride_1)/stride_1)*depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_hidden2], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))
    layer5_weights = tf.Variable(tf.truncated_normal(
        [num_hidden2, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
    def model_train(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, stride_1, stride_1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        drop = tf.nn.dropout(hidden, 0.75)
        conv = tf.nn.conv2d(drop, layer2_weights, [1, stride_2, stride_2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        drop = tf.nn.dropout(hidden, 0.75)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        drop = tf.nn.dropout(hidden, 0.75)
        hidden = tf.nn.relu(tf.matmul(drop, layer4_weights) + layer4_biases)
        return tf.matmul(hidden, layer5_weights) + layer5_biases

    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, stride_1, stride_1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, stride_2, stride_2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
        return tf.matmul(hidden, layer5_weights) + layer5_biases

  # Training computation.
    logits = model_train(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits, tf_train_labels) + \
        beta_regul *(
            1 * tf.nn.l2_loss(layer1_weights) + \
            1 * tf.nn.l2_loss(layer2_weights) + \
            1 * tf.nn.l2_loss(layer3_weights) + \
            1 * tf.nn.l2_loss(layer4_weights) + \
            1 * tf.nn.l2_loss(layer5_weights)))
     
  # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.25, global_step, 5000, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.sigmoid(logits)
    valid_prediction = tf.sigmoid(model(tf_valid_dataset))
    test_prediction = tf.sigmoid(model(tf_test_dataset))

	 minibatch	 minibatch	 validation
Steps	 loss     	 accuracy 	 accuracy
---------------------------------------------------
0	 1.036431	 52.759 %	 50.663 %	
500	 0.683988	 62.069 %	 60.486 %	
1000	 0.624465	 65.862 %	 71.129 %	
1500	 0.633703	 68.276 %	 78.105 %	
2000	 0.470493	 78.621 %	 84.543 %	
2500	 0.467052	 81.724 %	 80.426 %	
3000	 0.340815	 85.517 %	 87.535 %	
3500	 0.343168	 87.241 %	 88.155 %	
4000	 0.281665	 90.000 %	 88.222 %	
4500	 0.316459	 89.310 %	 89.223 %	
5000	 0.306602	 88.621 %	 89.651 %	
5500	 0.323350	 85.517 %	 89.656 %	
6000	 0.370277	 84.828 %	 88.278 %	
6500	 0.326758	 86.897 %	 89.764 %	
7000	 0.319943	 87.586 %	 90.069 %	
7500	 0.260177	 90.345 %	 90.058 %	
8000	 0.330911	 87.931 %	 89.795 %	
8500	 0.266430	 91.034 %	 90.378 %	
9000	 0.265763	 90.000 %	 90.528 %	
9500	 0.285275	 91.724 %	 90.358 %	
10000	 0.260739	 90.690 %	 90.487 %	
10500	 0.273746	 89.655 %	 89.919 %	
11000	 0.281265	 91.034 %	 90.590 %	
11500	 0.278727	 91.034 %	 90.512 %	
12000	 0.31

test accuracy : 90.517 %


In [6]:
num_steps = 15001
interval = 100
y = []
z = []

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(graph=graph, config=config) as session:
    tf.initialize_all_variables().run()
    print('\t', 'minibatch\t', 'minibatch\t', 'validation')
    print('Steps\t', 'loss     \t', 'accuracy \t', 'accuracy')
    print('---------------------------------------------------')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('%d\t %f\t %.3f %%\t %.3f %%\t' % (
                step,
                l,
                accuracy(predictions, batch_labels),
                accuracy(valid_prediction.eval(), valid_labels)
                ))
        if (step % interval == 0):
            y.append(accuracy(predictions, batch_labels))
            z.append(accuracy(valid_prediction.eval(), valid_labels))
    
    x = np.arange(0, num_steps, interval)
    plt.plot(x, y, 'r', label='train')
    plt.plot(x, z, 'b', label='validation')
    plt.xlabel("step")
    plt.ylabel("accuracy (%)")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
    plt.show()
    
    # Prediction

    with open('test_ans.data.txt', 'r') as f:
        i = 0
        count = 0
        result = np.argmax(test_prediction.eval(), 1)
        for line in f:
            ans = line.split(" ")[-1].split("\n")[0]
            if result[i] == int(ans):
                count += 1
            i += 1
    print("test accuracy : %.3f %%"%(100*count/len(result)))